In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("train.csv")

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [6]:
data.shape

(889, 12)

In [9]:
data.drop(["Name","Ticket","Cabin"],axis=1,inplace=True)

In [33]:
# Encoding all the categorical value
from sklearn.preprocessing import LabelEncoder
b = LabelEncoder()
data["Sex"] = b.fit_transform(y=data["Sex"])
data["Embarked"] = b.fit_transform(data["Embarked"])

In [39]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,2
1,2,1,1,0,38.0,1,0,71.2833,0
2,3,1,3,0,26.0,0,0,7.9250,2
3,4,1,1,0,35.0,1,0,53.1000,2
4,5,0,3,1,35.0,0,0,8.0500,2


In [41]:
data1 = pd.get_dummies(data["Sex"],drop_first= True)
data2 = pd.get_dummies(data["Embarked"],drop_first =True)

In [43]:
data["Male"] = data1

In [51]:
survived = data[data["Survived"] == 1]
death = data[data["Survived"] != 1]

In [52]:
survived.shape

(340, 10)

In [53]:
death.shape

(549, 10)

In [54]:
x = data.drop("Survived",axis=1)
y = data["Survived"]

In [55]:
from imblearn.over_sampling import SMOTE

In [56]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,random_state=24,test_size = 0.2)

In [64]:
model1 = SMOTE(random_state= 24)
model1x,model1y = model1.fit_resample(xtrain,ytrain)
model1y

0      0
1      0
2      0
3      0
4      0
      ..
861    1
862    1
863    1
864    1
865    1
Name: Survived, Length: 866, dtype: int64

In [65]:
survived = model1y[model1y == 1]
death = model1y[model1y != 1]

In [67]:
survived.shape

(433,)

In [68]:
death.shape

(433,)

In [70]:
# We created the balanced dataset

In [75]:
model1x.drop("PassengerId",inplace=True,axis = 1)

In [78]:
xtrain = model1x

In [81]:
xtrain.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Male
0,3,0,18.0,0,0,6.7500,1,0
1,3,1,26.0,2,0,8.6625,2,1
2,3,0,30.5,0,0,7.7500,1,0
3,2,1,34.0,0,0,13.0000,2,1
4,3,1,45.0,0,0,8.0500,2,1


In [79]:
ytrain = model1y

In [80]:
from sklearn.ensemble import RandomForestClassifier

In [83]:
model2 = RandomForestClassifier(n_estimators=1000,max_depth=16,oob_score=True)

In [86]:
features = xtrain.columns.values.tolist()

In [87]:
features

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Male']

In [88]:
model2.fit(data[features],ytrain)

RandomForestClassifier(max_depth=16, n_estimators=1000, oob_score=True)

In [89]:
print(model2.oob_score_)

0.8060046189376443


In [93]:
for features,imp in zip(features,model2.feature_importances_):
    print(features,imp)

Pclass 0.09606065146985032
Sex 0.1515122783432282
Age 0.22358949698056907
SibSp 0.04475396286907901
Parch 0.03789504938501687
Fare 0.2472282067055774
Embarked 0.037557372378650776
Male 0.16140298186802843


In [94]:
from sklearn import tree
model3 = tree.DecisionTreeClassifier()
model3.fit(xtrain,ytrain)

DecisionTreeClassifier()

In [106]:
c =model3.predict(xtest)

In [98]:
xtest.drop("PassengerId",axis = 1 ,inplace=True)

C:\Users\Dell\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [110]:
(c == ytest).sum()

144

In [111]:
len(ytest)

178

In [112]:
144/178

0.8089887640449438

In [113]:
test_data = pd.read_csv("test.csv")